In [1]:
import pandas as pd
import numpy as np
import ast
import re
import sklearn
import six
from abc import ABCMeta
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from scipy import sparse
import warnings
warnings.filterwarnings('ignore')
import nltk
import six
import re
import gensim

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from abc import ABCMeta
from sklearn.pipeline import Pipeline
from sklearn.utils import check_X_y, check_array
from scipy import sparse
from nltk.stem import PorterStemmer,WordNetLemmatizer
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.utils import resample
from sklearn.metrics import accuracy_score,f1_score

In [2]:
tsel = pd.read_csv(r'Prepro_Sinyal.csv', usecols=['Username','Text','Sinyal','text_result'])
tsel

,Username,Text,Sinyal,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,"['handphone', 'lapor', 'wajib', 'bantu', 'laca..."
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",-1.0,"['tidak tahu', 'telkomsel', 'lambat', 'kuota',..."
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,-1.0,"['kendala', 'sinyal', 'telkomsel']"
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,0.0,"['tidak bisa', 'nomor', 'mati', 'dihidupakan',..."
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",0.0,"['admin', 'kemarin', 'aplikasi', 'punya', 'tel..."
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",-1.0,"['sinyal', 'minus', 'tolong', 'baik', 'problem..."
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,-1.0,"['punya', 'telkomsel', 'ganggu', 'aduh', 'desa..."
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,"['allah', 'tambah', 'kerja', 'esuk', 'neng', '..."
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,-1.0,"['admin', 'sinyal', 'telkomsel', 'rumah', 'dae..."


In [3]:
tsel['text_result'] = tsel['text_result'].apply(str)

In [4]:
import ast

def join_text_list(text):
    texts = ast.literal_eval(text)
    return ' '.join([text for text in texts])

tsel['text_result'] = tsel['text_result'].apply(join_text_list)
tsel

,Username,Text,Sinyal,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,handphone lapor wajib bantu lacak imei perihal...
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",-1.0,tidak tahu telkomsel lambat kuota promo murah ...
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,-1.0,kendala sinyal telkomsel
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,0.0,tidak bisa nomor mati dihidupakan grapari nomo...
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",0.0,admin kemarin aplikasi punya telkomsel ganggu ...
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",-1.0,sinyal minus tolong baik problem location cama...
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,-1.0,punya telkomsel ganggu aduh desak pakai
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,allah tambah kerja esuk neng grapari
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,-1.0,admin sinyal telkomsel rumah daerah rawabunga ...


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
matrix = CountVectorizer(max_features=5000)
vectors = matrix.fit_transform(tsel['text_result']).toarray()

In [6]:
tsel['Sinyal'].value_counts()

-1.0    9643
 0.0    6877
 1.0     471
Name: Sinyal, dtype: int64

In [7]:
X = vectors
y = tsel['Sinyal']

X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [31]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=45)

## Klasifikasi NBSVM

In [32]:
class NBSVM(six.with_metaclass(ABCMeta, BaseEstimator, ClassifierMixin)):
    def __init__(self, alpha=1.0, C=1.0, max_iter=1000):
        self.alpha = alpha
        self.max_iter = max_iter
        self.C = C
        self.svm_ = [] # fuggly

    def fit(self, X, y):
        X, y = check_X_y(X, y, 'csr')
        _, n_features = X.shape

        labelbin = LabelBinarizer()
        Y = labelbin.fit_transform(y)
        self.classes_ = labelbin.classes_
        if Y.shape[1] == 1:
            Y = np.concatenate((1 - Y, Y), axis=1)

        # LabelBinarizer().fit_transform() returns arrays with dtype=np.int64.
        # so we don't have to cast X to floating point
        Y = Y.astype(np.float64)

        # Count raw events from data
        n_effective_classes = Y.shape[1]
        self.class_count_ = np.zeros(n_effective_classes, dtype=np.float64)
        self.ratios_ = np.full((n_effective_classes, n_features), self.alpha,
                                 dtype=np.float64)
        self._compute_ratios(X, Y)

        # flugglyness
        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            svm = LinearSVC(C=self.C, max_iter=self.max_iter)
            Y_i = Y[:,i]
            svm.fit(X_i, Y_i)
            self.svm_.append(svm) 

        return self

    def predict(self, X):
        n_effective_classes = self.class_count_.shape[0]
        n_examples = X.shape[0]

        D = np.zeros((n_effective_classes, n_examples))

        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            D[i] = self.svm_[i].decision_function(X_i)
        
        return self.classes_[np.argmax(D, axis=0)]
        
    def _compute_ratios(self, X, Y):
        """Count feature occurrences and compute ratios."""
        if np.any((X.data if issparse(X) else X) < 0):
            raise ValueError("Input X must be non-negative")

        self.ratios_ += safe_sparse_dot(Y.T, X)  # ratio + feature_occurrance_c
        normalize(self.ratios_, norm='l1', axis=1, copy=False)
        row_calc = lambda r: np.log(np.divide(r, (1 - r)))
        self.ratios_ = np.apply_along_axis(row_calc, axis=1, arr=self.ratios_)
        check_array(self.ratios_)
        self.ratios_ = sparse.csr_matrix(self.ratios_)

In [33]:
x_train_csr = sparse.csr_matrix(x_train)
x_test_csr = sparse.csr_matrix(x_test)
model = NBSVM()

model.fit(x_train_csr,y_train)
y_pred = model.predict(x_test_csr)
y_true = y_test

### Random 5

In [14]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[1739  124   46]
 [ 153 1227   14]
 [  48   10   38]]
              precision    recall  f1-score   support

        -1.0       0.90      0.91      0.90      1909
         0.0       0.90      0.88      0.89      1394
         1.0       0.39      0.40      0.39        96

    accuracy                           0.88      3399
   macro avg       0.73      0.73      0.73      3399
weighted avg       0.88      0.88      0.88      3399

Precision:  0.7285632792395992
Recall:  0.7289941115177029
F1:  0.7287026688568482
Accuracy:  0.8837893498087673


In [15]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.8278474639532444

### Pengujian random 15

In [19]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[1767  116   44]
 [ 143 1217   14]
 [  48   18   32]]
              precision    recall  f1-score   support

        -1.0       0.90      0.92      0.91      1927
         0.0       0.90      0.89      0.89      1374
         1.0       0.36      0.33      0.34        98

    accuracy                           0.89      3399
   macro avg       0.72      0.71      0.71      3399
weighted avg       0.89      0.89      0.89      3399

Precision:  0.7196070827845876
Recall:  0.7097450249209681
F1:  0.7144296835805717
Accuracy:  0.8873197999411592


In [20]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.8192893945453438

### Pengujian random 25

In [24]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[1776  110   43]
 [ 132 1226   17]
 [  50   12   33]]
              precision    recall  f1-score   support

        -1.0       0.91      0.92      0.91      1929
         0.0       0.91      0.89      0.90      1375
         1.0       0.35      0.35      0.35        95

    accuracy                           0.89      3399
   macro avg       0.72      0.72      0.72      3399
weighted avg       0.89      0.89      0.89      3399

Precision:  0.7237940889368158
Recall:  0.7198963590228221
F1:  0.7217855087039081
Accuracy:  0.8929096793174464


In [25]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.8262405111869411

### Pengujian random 35

In [29]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[1764  135   50]
 [ 133 1213   11]
 [  43   12   38]]
              precision    recall  f1-score   support

        -1.0       0.91      0.91      0.91      1949
         0.0       0.89      0.89      0.89      1357
         1.0       0.38      0.41      0.40        93

    accuracy                           0.89      3399
   macro avg       0.73      0.74      0.73      3399
weighted avg       0.89      0.89      0.89      3399

Precision:  0.72834283301991
Recall:  0.735855081730641
F1:  0.731967997060329
Accuracy:  0.8870255957634599


In [30]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.8326243978941322

### random 45

In [34]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[1755  130   57]
 [ 121 1217   16]
 [  53   18   32]]
              precision    recall  f1-score   support

        -1.0       0.91      0.90      0.91      1942
         0.0       0.89      0.90      0.90      1354
         1.0       0.30      0.31      0.31       103

    accuracy                           0.88      3399
   macro avg       0.70      0.70      0.70      3399
weighted avg       0.88      0.88      0.88      3399

Precision:  0.7020449396810206
Recall:  0.7044018152578618
F1:  0.7032056012434529
Accuracy:  0.8837893498087673


In [35]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.8165437127483083